In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from pgmpy.estimators import K2Score, BicScore, BDeuScore
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BayesianEstimator

In [ ]:
offense = pd.read_csv('NYPD_Complaint_Data_Current__Year_To_Date_.csv')
offmap = gpd.read_file('NYPD_Complaint_Map__Year_to_Date_.csv')
offmap.crs = 'epsg:4326'

In [ ]:
offense.columns.unique()

Index(['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM',
       'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT',
       'HOUSING_PSA', 'JURISDICTION_CODE', 'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD',
       'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PATROL_BORO', 'PD_CD',
       'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT', 'STATION_NAME', 'SUSP_AGE_GROUP',
       'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'VIC_AGE_GROUP',
       'VIC_RACE', 'VIC_SEX', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude',
       'Longitude', 'Lat_Lon', 'New Georeferenced Column'],
      dtype='object')

In [ ]:
offense1 = offense[['BORO_NM','CMPLNT_FR_DT','CMPLNT_FR_TM','LAW_CAT_CD','OFNS_DESC','SUSP_RACE','Lat_Lon']]
offense1 = offense1.dropna()

In [ ]:
offmap.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,JURIS_DESC,...,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,Lat_Lon,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,geometry
0,903695881,69,,12/17/2021,22:13:00,,,COMPLETED,,N.Y. POLICE DEPT,...,,,,12/17/2021,POINT (-73.90287588699994 40.64564719600002),1011203,174515,40.64564719600002,-73.90287588699994,None
1,400462399,113,,12/17/2021,06:21:00,,,COMPLETED,,N.Y. POLICE DEPT,...,,,,12/17/2021,POINT (-73.78726915499993 40.682499421000045),1043252,187998,40.682499421000045,-73.78726915499993,None
2,587910690,113,,12/13/2021,20:05:00,,,COMPLETED,,N.Y. POLICE DEPT,...,,,,12/13/2021,POINT (-73.79144856999993 40.689218058000044),1042087,190443,40.689218058000044,-73.79144856999993,None
3,186105368,52,BRONX,12/07/2021,22:49:00,,,COMPLETED,,N.Y. POLICE DEPT,...,792,WEAPONS POSSESSION 1 & 2,STREET,12/07/2021,POINT (-73.88124724999993 40.88271780700006),1017088,260895,40.88271780700006,-73.88124724999993,None
4,185325394,113,,12/06/2021,17:25:00,,,COMPLETED,,N.Y. POLICE DEPT,...,,,,12/06/2021,POINT (-73.77667979199998 40.69648322200004),1046176,193100,40.69648322200004,-73.77667979199998,None


In [ ]:
offense1.SUSP_RACE.unique()

array(['UNKNOWN', 'BLACK', 'BLACK HISPANIC', 'WHITE', 'WHITE HISPANIC',
       'ASIAN / PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKAN NATIVE'],
      dtype=object)

In [ ]:
offense1 = offense1[offense1.SUSP_RACE != 'UNKNOWN']

In [ ]:
offense1.SUSP_RACE.unique()

array(['BLACK', 'BLACK HISPANIC', 'WHITE', 'WHITE HISPANIC',
       'ASIAN / PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKAN NATIVE'],
      dtype=object)

In [ ]:
offense1['CMPLNT_FR_DT'] = pd.to_datetime(offense1['CMPLNT_FR_DT'], errors = 'coerce')
offense1['year'] = offense1['CMPLNT_FR_DT'].dt.year
offense1['day'] = offense1['CMPLNT_FR_DT'].dt.dayofweek
offense1 = offense1.dropna()

C:\Users\13502\AppData\Local\Temp\ipykernel_576\2210413022.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offense1['CMPLNT_FR_DT'] = pd.to_datetime(offense1['CMPLNT_FR_DT'], errors = 'coerce')
C:\Users\13502\AppData\Local\Temp\ipykernel_576\2210413022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offense1['year'] = offense1['CMPLNT_FR_DT'].dt.year
C:\Users\13502\AppData\Local\Temp\ipykernel_576\2210413022.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [ ]:
offense1.head()

,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,SUSP_RACE,Lat_Lon,year,day
71,BRONX,2021-12-31,04:20:00,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,BLACK,"(40.838689661000046, -73.91880635699994)",2021.0,4.0
74,BROOKLYN,2021-12-22,10:35:00,FELONY,MISCELLANEOUS PENAL LAW,BLACK,"(40.66456888000005, -73.88691987599998)",2021.0,2.0
75,BRONX,2021-12-27,13:18:00,VIOLATION,HARRASSMENT 2,BLACK HISPANIC,"(40.84734979900003, -73.90189464799995)",2021.0,0.0
76,QUEENS,2021-12-26,19:00:00,FELONY,GRAND LARCENY,WHITE,"(40.68321413800004, -73.74215413399996)",2021.0,6.0
77,BRONX,2021-12-25,17:00:00,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,WHITE HISPANIC,"(40.85026097600007, -73.88874043199996)",2021.0,5.0


In [ ]:
offense1['CMPLNT_FR_TM'] = pd.to_datetime(offense1['CMPLNT_FR_TM'])

C:\Users\13502\AppData\Local\Temp\ipykernel_576\2081916888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offense1['CMPLNT_FR_TM'] = pd.to_datetime(offense1['CMPLNT_FR_TM'])


In [ ]:
offense1['hour'] = offense1['CMPLNT_FR_TM'].dt.hour

C:\Users\13502\AppData\Local\Temp\ipykernel_576\442718291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offense1['hour'] = offense1['CMPLNT_FR_TM'].dt.hour


In [ ]:
offense1 = offense1.drop(columns = ['CMPLNT_FR_TM'])

In [ ]:
offense1.LAW_CAT_CD.unique()

array(['MISDEMEANOR', 'FELONY', 'VIOLATION'], dtype=object)

In [ ]:
offense1.SUSP_RACE.unique()

array(['BLACK', 'BLACK HISPANIC', 'WHITE', 'WHITE HISPANIC',
       'ASIAN / PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKAN NATIVE'],
      dtype=object)

### Cluster the number of offense in different day of week

In [ ]:
offasian = offense1[offense1['SUSP_RACE'] =='ASIAN / PACIFIC ISLANDER']
dfasian = offasian.groupby(by = ['day','OFNS_DESC']).agg({'OFNS_DESC':'count'})

In [ ]:
dfasian = dfasian.rename(columns = {'OFNS_DESC':'num'})
dfasian.reset_index(inplace = True)
dfasian.head()

,day,OFNS_DESC,num
0,0.0,ADMINISTRATIVE CODE,15
1,0.0,ALCOHOLIC BEVERAGE CONTROL LAW,1
2,0.0,ARSON,1
3,0.0,ASSAULT 3 & RELATED OFFENSES,348
4,0.0,BURGLAR'S TOOLS,1


In [ ]:
tableasian = pd.pivot_table(dfasian,values ='num',index = 'OFNS_DESC',columns = 'day')
tableasian = tableasian.fillna(0)
tableasian.head()

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0
OFNS_DESC,,,,,,,
ADMINISTRATIVE CODE,15.0,12.0,8.0,10.0,7.0,11.0,8.0
AGRICULTURE & MRKTS LAW-UNCLASSIFIED,0.0,1.0,0.0,1.0,0.0,0.0,0.0
ALCOHOLIC BEVERAGE CONTROL LAW,1.0,1.0,1.0,1.0,1.0,1.0,0.0
ARSON,1.0,2.0,0.0,1.0,0.0,0.0,0.0
ASSAULT 3 & RELATED OFFENSES,348.0,320.0,323.0,326.0,320.0,373.0,370.0


In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:

    clusterer = KMeans(n_clusters=n_clusters, random_state=0).fit(tableasian)

    silhouette_avg = silhouette_score(tableasian, clusterer.labels_)
    
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.wa

For n_clusters = 2 The average silhouette_score is : 0.8836505554077915
For n_clusters = 3 The average silhouette_score is : 0.7673076233704272
For n_clusters = 4 The average silhouette_score is : 0.7510715162476919
For n_clusters = 5 The average silhouette_score is : 0.6301136752931635
For n_clusters = 6 The average silhouette_score is : 0.6272224608063637


C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
km=KMeans(random_state=0,n_clusters=2)
cluster_labels = km.fit_predict(tableasian)
tableasian['labelk']=cluster_labels

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
tableasian['labelk'].value_counts()

0    40
1     2
Name: labelk, dtype: int64

In [ ]:
tableasian[tableasian['labelk']==1]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk
OFNS_DESC,,,,,,,,
ASSAULT 3 & RELATED OFFENSES,348.0,320.0,323.0,326.0,320.0,373.0,370.0,1
HARRASSMENT 2,556.0,520.0,519.0,528.0,568.0,524.0,550.0,1


In [ ]:
from sklearn.mixture import GaussianMixture 

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:

    clusterer = GaussianMixture(n_components=n_clusters, random_state=0).fit_predict(tableasian)

    silhouette_avg = silhouette_score(tableasian, clusterer)
    
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

For n_clusters = 2 The average silhouette_score is : 0.8836506219829577
For n_clusters = 3 The average silhouette_score is : 0.7673076546758082
For n_clusters = 4 The average silhouette_score is : 0.7510715162476919
For n_clusters = 5 The average silhouette_score is : 0.6360973300147874
For n_clusters = 6 The average silhouette_score is : 0.5729489764251983


C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be 

In [ ]:
clusterer = GaussianMixture(n_components=2, random_state=0).fit_predict(tableasian)
tableasian['labelg']=clusterer
tableasian['labelg'].value_counts()

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(


0    40
1     2
Name: labelg, dtype: int64

In [ ]:
tableasian[tableasian['labelg']==1]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk,labelg
OFNS_DESC,,,,,,,,,
ASSAULT 3 & RELATED OFFENSES,348.0,320.0,323.0,326.0,320.0,373.0,370.0,1,1
HARRASSMENT 2,556.0,520.0,519.0,528.0,568.0,524.0,550.0,1,1


In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster
range_n_clusters = range(2,7)
Z = linkage(tableasian, 'single')

for n_clusters in range_n_clusters:
    cluster_labels=fcluster(Z, n_clusters, criterion='maxclust')    
    silhouette_avg = silhouette_score(tableasian, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

For n_clusters = 2, the average silhouette_score is : 0.8579962768499437
For n_clusters = 3, the average silhouette_score is : 0.8266106449835406
For n_clusters = 4, the average silhouette_score is : 0.6924201329826066
For n_clusters = 5, the average silhouette_score is : 0.7115836151456172
For n_clusters = 6, the average silhouette_score is : 0.6861886501818715


In [ ]:
Z = linkage(tableasian, 'single')
cluster_labels=fcluster(Z, 2, criterion='maxclust')
tableasian['labels']=cluster_labels
tableasian['labels'].value_counts()

1    41
2     1
Name: labels, dtype: int64

In [ ]:
tableasian[tableasian['labels']==2]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk,labelg,labels
OFNS_DESC,,,,,,,,,,
HARRASSMENT 2,556.0,520.0,519.0,528.0,568.0,524.0,550.0,1,1,2


In [ ]:
range_n_clusters = range(2,7)
Z = linkage(tableasian, 'complete')

for n_clusters in range_n_clusters:
    cluster_labels=fcluster(Z, n_clusters, criterion='maxclust')    
    silhouette_avg = silhouette_score(tableasian, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

For n_clusters = 2, the average silhouette_score is : 0.8836506666791225
For n_clusters = 3, the average silhouette_score is : 0.8266106449835406
For n_clusters = 4, the average silhouette_score is : 0.7412791488116665
For n_clusters = 5, the average silhouette_score is : 0.7101256797303146
For n_clusters = 6, the average silhouette_score is : 0.6500902348678571


In [ ]:
Z = linkage(tableasian, 'complete')
cluster_labels=fcluster(Z, 2, criterion='maxclust')
tableasian['labelc']=cluster_labels
tableasian['labelc'].value_counts()

1    40
2     2
Name: labelc, dtype: int64

In [ ]:
tableasian[tableasian['labelc']==2]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk,labelg,labels,labelc
OFNS_DESC,,,,,,,,,,,
ASSAULT 3 & RELATED OFFENSES,348.0,320.0,323.0,326.0,320.0,373.0,370.0,1,1,1,2
HARRASSMENT 2,556.0,520.0,519.0,528.0,568.0,524.0,550.0,1,1,2,2


### hispanic

In [ ]:
offbhis = offense1[(offense1['SUSP_RACE'] =='BLACK HISPANIC')]
offwhis = offense1[(offense1['SUSP_RACE'] =='WHITE HISPANIC')]
offhis = pd.concat([offbhis,offwhis])
dfhis = offhis.groupby(by = ['day','OFNS_DESC']).agg({'OFNS_DESC':'count'})

In [ ]:
dfhis = dfhis.rename(columns = {'OFNS_DESC':'num'})
dfhis.reset_index(inplace = True)
dfhis.head()

,day,OFNS_DESC,num
0,0.0,ADMINISTRATIVE CODE,24
1,0.0,AGRICULTURE & MRKTS LAW-UNCLASSIFIED,1
2,0.0,ALCOHOLIC BEVERAGE CONTROL LAW,1
3,0.0,ANTICIPATORY OFFENSES,3
4,0.0,ARSON,4


In [ ]:
table = pd.pivot_table(dfhis,values ='num',index = 'OFNS_DESC',columns = 'day')
table = table.fillna(0)
table.head()

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0
OFNS_DESC,,,,,,,
ADMINISTRATIVE CODE,24.0,22.0,15.0,14.0,22.0,16.0,12.0
AGRICULTURE & MRKTS LAW-UNCLASSIFIED,1.0,1.0,4.0,1.0,0.0,0.0,0.0
ALCOHOLIC BEVERAGE CONTROL LAW,1.0,0.0,0.0,5.0,1.0,0.0,1.0
ANTICIPATORY OFFENSES,3.0,0.0,1.0,2.0,0.0,0.0,0.0
ARSON,4.0,5.0,2.0,3.0,2.0,2.0,5.0


In [ ]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:

    clusterer = KMeans(n_clusters=n_clusters, random_state=0).fit(table)

    silhouette_avg = silhouette_score(table, clusterer.labels_)
    
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.wa

For n_clusters = 2 The average silhouette_score is : 0.8772676266300368
For n_clusters = 3 The average silhouette_score is : 0.7844388702860965
For n_clusters = 4 The average silhouette_score is : 0.7687319197245733
For n_clusters = 5 The average silhouette_score is : 0.7116467302733727
For n_clusters = 6 The average silhouette_score is : 0.7075921288542075


In [ ]:
km=KMeans(random_state=0,n_clusters=2)
cluster_labels = km.fit_predict(table)
table['labelk']=cluster_labels
table['labelk'].value_counts()

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  warnings.warn(


0    50
1     3
Name: labelk, dtype: int64

In [ ]:
table[table['labelk']==1]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk
OFNS_DESC,,,,,,,,
ASSAULT 3 & RELATED OFFENSES,1760.0,1595.0,1605.0,1654.0,1885.0,2074.0,2280.0,1
HARRASSMENT 2,2237.0,2273.0,2254.0,2260.0,2235.0,2318.0,2245.0,1
PETIT LARCENY,1351.0,1463.0,1491.0,1374.0,1383.0,1183.0,1209.0,1


In [ ]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:

    clusterer = GaussianMixture(n_components=n_clusters, random_state=0).fit_predict(table)

    silhouette_avg = silhouette_score(table, clusterer)
    
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

For n_clusters = 2 The average silhouette_score is : 0.8772676310769677
For n_clusters = 3 The average silhouette_score is : 0.7844388723477556
For n_clusters = 4 The average silhouette_score is : 0.7687319221525376
For n_clusters = 5 The average silhouette_score is : 0.7564703747339486
For n_clusters = 6 The average silhouette_score is : 0.7291294715393845


C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be 

In [ ]:
clusterer = GaussianMixture(n_components=2, random_state=0).fit_predict(table)
table['labelg']=clusterer
table['labelg'].value_counts()

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  warnings.warn(


0    50
1     3
Name: labelg, dtype: int64

In [ ]:
table[table['labelg']==1]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk,labelg
OFNS_DESC,,,,,,,,,
ASSAULT 3 & RELATED OFFENSES,1760.0,1595.0,1605.0,1654.0,1885.0,2074.0,2280.0,1,1
HARRASSMENT 2,2237.0,2273.0,2254.0,2260.0,2235.0,2318.0,2245.0,1,1
PETIT LARCENY,1351.0,1463.0,1491.0,1374.0,1383.0,1183.0,1209.0,1,1


In [ ]:
range_n_clusters = range(2,7)
Z = linkage(table, 'single')

for n_clusters in range_n_clusters:
    cluster_labels=fcluster(Z, n_clusters, criterion='maxclust')    
    silhouette_avg = silhouette_score(table, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

For n_clusters = 2, the average silhouette_score is : 0.8772676355238984
For n_clusters = 3, the average silhouette_score is : 0.8107585384618256
For n_clusters = 4, the average silhouette_score is : 0.7984969910432367
For n_clusters = 5, the average silhouette_score is : 0.698363008296472
For n_clusters = 6, the average silhouette_score is : 0.5117922538646411


In [ ]:
Z = linkage(table, 'single')
cluster_labels=fcluster(Z, 2, criterion='maxclust')
table['labels']=cluster_labels
table['labels'].value_counts()

1    50
2     3
Name: labels, dtype: int64

In [ ]:
table[table['labels']==2]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk,labelg,labels
OFNS_DESC,,,,,,,,,,
ASSAULT 3 & RELATED OFFENSES,1760.0,1595.0,1605.0,1654.0,1885.0,2074.0,2280.0,1,1,2
HARRASSMENT 2,2237.0,2273.0,2254.0,2260.0,2235.0,2318.0,2245.0,1,1,2
PETIT LARCENY,1351.0,1463.0,1491.0,1374.0,1383.0,1183.0,1209.0,1,1,2


In [ ]:
range_n_clusters = range(2,7)
Z = linkage(table, 'complete')

for n_clusters in range_n_clusters:
    cluster_labels=fcluster(Z, n_clusters, criterion='maxclust')    
    silhouette_avg = silhouette_score(table, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

For n_clusters = 2, the average silhouette_score is : 0.8772676399708284
For n_clusters = 3, the average silhouette_score is : 0.8107585493066448
For n_clusters = 4, the average silhouette_score is : 0.768731927008462
For n_clusters = 5, the average silhouette_score is : 0.7413910189579737
For n_clusters = 6, the average silhouette_score is : 0.7291294715393845


In [ ]:
Z = linkage(table, 'complete')
cluster_labels=fcluster(Z, 2, criterion='maxclust')
table['labelc']=cluster_labels
table['labelc'].value_counts()

1    50
2     3
Name: labelc, dtype: int64

In [ ]:
table[table['labelc']==2]

day,0.0,1.0,2.0,3.0,4.0,5.0,6.0,labelk,labelg,labels,labelc
OFNS_DESC,,,,,,,,,,,
ASSAULT 3 & RELATED OFFENSES,1760.0,1595.0,1605.0,1654.0,1885.0,2074.0,2280.0,1,1,2,2
HARRASSMENT 2,2237.0,2273.0,2254.0,2260.0,2235.0,2318.0,2245.0,1,1,2,2
PETIT LARCENY,1351.0,1463.0,1491.0,1374.0,1383.0,1183.0,1209.0,1,1,2,2


### Bayesian Network

In [ ]:
offense2 = offense[['BORO_NM','OFNS_DESC','SUSP_SEX','SUSP_RACE']]
offense2 = offense2[offense2.SUSP_RACE != 'UNKNOWN']
offense2 = offense2.dropna()
offense2.head()

,BORO_NM,OFNS_DESC,SUSP_SEX,SUSP_RACE
71,BRONX,ASSAULT 3 & RELATED OFFENSES,M,BLACK
74,BROOKLYN,MISCELLANEOUS PENAL LAW,M,BLACK
75,BRONX,HARRASSMENT 2,M,BLACK HISPANIC
76,QUEENS,GRAND LARCENY,F,WHITE
77,BRONX,ASSAULT 3 & RELATED OFFENSES,M,WHITE HISPANIC


In [ ]:
from sklearn import preprocessing
off3 = offense2.copy()
le = preprocessing.LabelEncoder()
off3.BORO_NM = le.fit_transform(off3.BORO_NM)
off3.OFNS_DESC = le.fit_transform(off3.OFNS_DESC)
off3.SUSP_SEX = le.fit_transform(off3.SUSP_SEX)
off3.SUSP_RACE = le.fit_transform(off3.SUSP_RACE)
off3.head()

,BORO_NM,OFNS_DESC,SUSP_SEX,SUSP_RACE
71,0,5,1,2
74,1,34,1,2
75,0,25,1,3
76,3,23,0,4
77,0,5,1,5


In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(off3,random_state=9,test_size=0.4)
hc = HillClimbSearch(train)
best_model = hc.estimate(scoring_method=BicScore(train))
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('BORO_NM', 'SUSP_RACE'), ('BORO_NM', 'OFNS_DESC'), ('OFNS_DESC', 'SUSP_SEX')]


In [ ]:
model = BayesianModel([('BORO_NM', 'SUSP_RACE'), ('BORO_NM', 'OFNS_DESC'), ('OFNS_DESC', 'SUSP_SEX')])
model.fit(train, estimator=BayesianEstimator, prior_type='K2')
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of BORO_NM:
+------------+-----------+
| BORO_NM(0) | 0.215094  |
+------------+-----------+
| BORO_NM(1) | 0.271271  |
+------------+-----------+
| BORO_NM(2) | 0.265733  |
+------------+-----------+
| BORO_NM(3) | 0.206857  |
+------------+-----------+
| BORO_NM(4) | 0.0410454 |
+------------+-----------+
CPD of SUSP_RACE:
+--------------+----------------------+-----+-----------------------+----------------------+
| BORO_NM      | BORO_NM(0)           | ... | BORO_NM(3)            | BORO_NM(4)           |
+--------------+----------------------+-----+-----------------------+----------------------+
| SUSP_RACE(0) | 0.003043289236693373 | ... | 0.0034550679711970035 | 0.004878842088144414 |
+--------------+----------------------+-----+-----------------------+----------------------+
| SUSP_RACE(1) | 0.015837525619526737 | ... | 0.14311085278827215   | 0.027972027972027972 |
+--------------+----------------------+-----+-----------------------+----------------------+
| SUSP_RACE(2) | 0

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [ ]:
print(model.get_cpds('SUSP_RACE'))

+--------------+----------------------+-----+-----------------------+----------------------+
| BORO_NM      | BORO_NM(0)           | ... | BORO_NM(3)            | BORO_NM(4)           |
+--------------+----------------------+-----+-----------------------+----------------------+
| SUSP_RACE(0) | 0.003043289236693373 | ... | 0.0034550679711970035 | 0.004878842088144414 |
+--------------+----------------------+-----+-----------------------+----------------------+
| SUSP_RACE(1) | 0.015837525619526737 | ... | 0.14311085278827215   | 0.027972027972027972 |
+--------------+----------------------+-----+-----------------------+----------------------+
| SUSP_RACE(2) | 0.5075771691199304   | ... | 0.4155445768348994    | 0.38396487233696536  |
+--------------+----------------------+-----+-----------------------+----------------------+
| SUSP_RACE(3) | 0.13716539345382275  | ... | 0.03913591010365204   | 0.03675394373068792  |
+--------------+----------------------+-----+-----------------------+-

In [ ]:
from sklearn.metrics import accuracy_score
print(test.head())
apred = model.predict(test.loc[:,["BORO_NM"]])

        BORO_NM  OFNS_DESC  SUSP_SEX  SUSP_RACE
315058        2          5         1          2
368061        2         47         1          2
199145        3         53         1          5
130980        1         47         0          2
142722        1          5         1          5


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\13502\anaconda3\envs\geo_env\lib\site-packages\pgmpy\models\BayesianNetwork.py:747: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return data.merge(data_with_results, how="left").loc[:, missing_variables]


In [ ]:
apred

,SUSP_RACE,OFNS_DESC,SUSP_SEX
0,2,47,1
1,2,47,1
2,2,25,1
3,2,25,1
4,2,25,1
...,...,...,...
99785,2,25,1
99786,2,25,1
99787,2,47,1
99788,2,47,1


In [ ]:
test.loc[:,['SUSP_RACE','OFNS_DESC','SUSP_SEX']]

,SUSP_RACE,OFNS_DESC,SUSP_SEX
315058,2,5,1
368061,2,47,1
199145,5,53,1
130980,2,47,0
142722,5,5,1
...,...,...,...
298350,2,17,2
132944,5,25,0
16634,4,23,1
328444,4,9,1


In [ ]:
print('out-of-sample accuracy for race is ',accuracy_score(test.iloc[:,0], apred.iloc[:,0]))
print('out-of-sample accuracy for offense type is ',accuracy_score(test.iloc[:,1], apred.iloc[:,1]))
print('out-of-sample accuracy for sex is ',accuracy_score(test.iloc[:,2], apred.iloc[:,2]))

out-of-sample accuracy for race is  0.2637037779336607
out-of-sample accuracy for offense type is  0.24439322577412567
out-of-sample accuracy for sex is  0.7556167952700671
